In [188]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from pyaxis import pyaxis
import matplotlib.font_manager as fm 
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import numpy as np

In [189]:
# Load the "Jost" font needed for this plot
font_dir = ['./Jost_Font/']  # Font is located in Jost_Font folder
for font in fm.findSystemFonts(font_dir):
    fm.fontManager.addfont(font)

In [190]:
gpd.options.io_engine = "pyogrio"

In [191]:
data_participation = pd.read_csv("./NRW2023-wahlbeteiligung.csv", sep=';')

In [192]:
data_language = pd.read_csv("./language_per_municipality.csv", sep=';')

In [193]:
data_municipality = data_participation[data_participation["ebene_resultat"].isin(["Gemeinde"])]

In [194]:
map_municipality = gpd.read_file("./boundaries/k4g23.shx")

In [195]:
data_municipality["kanton_abbr"] = data_municipality["kanton_nummer"].map({
    1: "ZH",
    2: "BE",
    3: "LU",
    4: "UR",
    5: "SZ",
    6: "OW",
    7: "NW",
    8: "GL",
    9: "ZG",
    10: "FR",
    11: "SO",
    12: "BS",
    13: "BL",
    14: "SH",
    15: "AR",
    16: "AI",
    17: "SG",
    18: "GR",
    19: "AG",
    20: "TG",
    21: "TI",
    22: "VD",
    23: "VS",
    24: "NE",
    25: "GE",
    26: "JU",
})

C:\Users\dave_\AppData\Local\Temp\ipykernel_17616\1805041338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_municipality["kanton_abbr"] = data_municipality["kanton_nummer"].map({


In [196]:
data_municipality = data_municipality.rename(columns={'geoLevelName': 'GEO_NAME'})
map_municipality = map_municipality.rename(columns={'GMDNAME': 'GEO_NAME'})
data_municipality = data_municipality.merge(data_language, on="GEO_NAME")
data_municipality = data_municipality.merge(map_municipality, on="GEO_NAME")

In [197]:
data_municipality["voter_per_qkm2"] = data_municipality["wahlberechtigte"].astype(int) / (data_municipality["AREA_HA"].astype(float) / 100)

In [198]:
from matplotlib.colors import LinearSegmentedColormap

cmap = LinearSegmentedColormap.from_list(name="my_cmap", 
                                                colors=["orange", "crimson", "darkgreen", "cornflowerblue"], N=4)

In [199]:
cmap = {
    'Deutsches Sprachgebiet': '#ffa500', 
    'Französisches Sprachgebiet': '#dc143c',
    'Italienisches Sprachgebiet': '#006400', 
    'Rätoromanisches Sprachgebiet': '#6495ed'
}

In [200]:
languages_separated = {}
for key in cmap.keys():
    languages_separated[key] = data_municipality[data_municipality["DESC_VAL"].isin([key])]

In [201]:
# Calculations for polynomial regression fitting

data_mean = {}
for key, df in languages_separated.items():
    # Fit a polynomial curve to the data
    degree = 1  # Eight degree seems to fit the original the best
    coefficients = np.polyfit(df['voter_per_qkm2'], df["wahlbeteiligung"], degree)
    polynomial = np.poly1d(coefficients)

    # Create x values for the smooth curve
    x_smooth = np.linspace(data_municipality['voter_per_qkm2'].min(), data_municipality['voter_per_qkm2'].max(), 1000)

    # Evaluate the polynomial at the x_smooth values to get the corresponding y_smooth values
    y_smooth = polynomial(x_smooth)

    # Store the curve data in new 'data_mean' dict
    data_mean[key] = (x_smooth, y_smooth)

In [202]:
# Create figure and axis
f, ax = plt.subplots(figsize=(36, 27), dpi=300)

ax.scatter(x='voter_per_qkm2', y='wahlbeteiligung', c=data_municipality['DESC_VAL'].map(cmap), data=data_municipality, s=150, edgecolors='white', zorder=90)
ax.set_xscale("log")

for key, df in data_mean.items():
    ax.plot(df[0], df[1], linewidth=5, color=cmap.get(key), zorder=99)
    ax.plot(df[0], df[1], linewidth=7, color='white', alpha=1, zorder=98)

ax.set_ylim(bottom=15, top=85)
ax.set_xlim(left=data_municipality['voter_per_qkm2'].min(), right=data_municipality['voter_per_qkm2'].max())


# ax.tick_params(which='both', bottom=False, top=False, right=False, left=False, labelleft=False, labelbottom=False) 

ax.spines.bottom.set_visible(False)
ax.spines.top.set_visible(False)
ax.spines.left.set_visible(False)
ax.spines.right.set_visible(False)

# do_annotations(ax)

# f.legend(handles=legend_elements, bbox_to_anchor=(0.322, 0.825), **legend_kwargs)

# Set title
f.suptitle('Voter Population Density on Municipality Level', fontproperties={'family': 'Jost', 'size': 60, 'weight': 'bold'}, x=0.405, y=0.9)

# Set subtitle
f.text(x=0.157, y=0.85, s=f"The density of voters per municipality in voters/km²", color='dimgrey', fontproperties={'family': 'Jost', 'size': 27, 'weight': 'regular'})

f.text(x=0.8, y=0.175, s="Sources: bfs.admin.ch", color='dimgrey', fontproperties={'family': 'Jost', 'size': 18, 'weight': 'semibold'})

# Save the figure -> the output "plot.png" looks a lot closer to the original than the one displayed in this notebook
plt.savefig('density_v_participation.png', bbox_inches='tight', pad_inches=0.5)